# Finetuning Ranker

# Organizando o ambiente

## Importações

In [5]:
from tqdm import tqdm

In [6]:
import io

In [7]:
import time


In [4]:
import seaborn as sns
import matplotlib.pyplot as plt

In [8]:
import string

In [9]:
import gzip

In [10]:
import getpass

In [11]:
import gc

In [12]:
import json, time

In [13]:
from transformers import BatchEncoding

/home/borela/miniconda3/envs/treinapython39/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [14]:
import torch

In [15]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification 
# para monot5: AutoModelForSeq2SeqLM

In [16]:
from torch.utils.data import DataLoader

In [17]:
from torch.utils.data import Dataset

In [18]:
from psutil import virtual_memory

In [19]:
import pandas as pd
import os

In [20]:
import random
import numpy as np
import torch

In [21]:
import transformers

In [22]:
import datasets

In [23]:
import pickle

In [24]:
from statistics import mean, stdev

## Definindo paths

In [25]:
PATH_TRAIN_DATA = '../data/train_juris_tcu_index/train_data_juris_tcu_index_bm25.csv'
PATH_TRAIN_MODEL =  '../model/juris_tcu_index'

In [55]:
PATH_QUERY = '../data/juris_tcu_index/query.csv'
PATH_DOC =  '../data/juris_tcu_index/doc.csv'

In [26]:
if os.path.exists(PATH_TRAIN_MODEL):
    print('pasta já existia!')
else:
    os.makedirs(PATH_TRAIN_MODEL)
    print('pasta criada!')


pasta criada!


Constants


In [90]:
MODEL_MAX_LENGTH = 512

## Função de verificação de memória

In [27]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Mon Jun 19 18:10:14 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.116.03   Driver Version: 525.116.03   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:02:00.0 Off |                  N/A |
|100%   80C    P2   306W / 370W |  22352MiB / 24576MiB |    100%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [28]:
def mostra_memoria(lista_mem=['cpu']):
  """
  Esta função exibe informações de memória da CPU e/ou GPU, conforme parâmetros fornecidos.

  Parâmetros:
  -----------
  lista_mem : list, opcional
      Lista com strings 'cpu' e/ou 'gpu'. 
      'cpu' - exibe informações de memória da CPU.
      'gpu' - exibe informações de memória da GPU (se disponível).
      O valor padrão é ['cpu'].

  Saída:
  -------
  A função não retorna nada, apenas exibe as informações na tela.

  Exemplo de uso:
  ---------------
  Para exibir informações de memória da CPU:
      mostra_memoria(['cpu'])

  Para exibir informações de memória da CPU e GPU:
      mostra_memoria(['cpu', 'gpu'])
  
  Autor: Marcus Vinícius Borela de Castro

  """  
  if 'cpu' in lista_mem:
    vm = virtual_memory()
    ram={}
    ram['total']=round(vm.total / 1e9,2)
    ram['available']=round(virtual_memory().available / 1e9,2)
    # ram['percent']=round(virtual_memory().percent / 1e9,2)
    ram['used']=round(virtual_memory().used / 1e9,2)
    ram['free']=round(virtual_memory().free / 1e9,2)
    ram['active']=round(virtual_memory().active / 1e9,2)
    ram['inactive']=round(virtual_memory().inactive / 1e9,2)
    ram['buffers']=round(virtual_memory().buffers / 1e9,2)
    ram['cached']=round(virtual_memory().cached/1e9 ,2)
    print(f"Your runtime RAM in gb: \n total {ram['total']}\n available {ram['available']}\n used {ram['used']}\n free {ram['free']}\n cached {ram['cached']}\n buffers {ram['buffers']}")
    print('/nGPU')
    gpu_info = !nvidia-smi
  if 'gpu' in lista_mem:
    gpu_info = '\n'.join(gpu_info)
    if gpu_info.find('failed') >= 0:
      print('Not connected to a GPU')
    else:
      print(gpu_info)


In [29]:
mostra_memoria(['cpu','gpu'])

Your runtime RAM in gb: 
 total 67.35
 available 47.86
 used 18.47
 free 19.64
 cached 27.89
 buffers 1.35
/nGPU
Mon Jun 19 18:10:20 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.116.03   Driver Version: 525.116.03   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:02:00.0 Off |                  N/A |
|100%   80C    P2   309W / 370W |  22352MiB / 24576MiB |    100%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------

## Definindo Hiperparâmetros iniciais

In [30]:
def inicia_hparam()->dict:
  # Inicialização dos parâmetros
  hparam = {}
  hparam["num_workers_dataloader"] = 0
  hparam["device"] = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
  if torch.cuda.is_available(): print(torch. cuda. get_device_name(hparam["device"]))    
  return hparam

In [31]:
hparam=inicia_hparam()

NVIDIA GeForce RTX 3090


In [32]:
hparam

{'num_workers_dataloader': 0, 'device': device(type='cuda', index=0)}

## Fixando as seeds

In [33]:
def inicializa_seed(num_semente:int=123):
  """
  Inicializa as sementes para garantir a reprodutibilidade dos resultados do modelo.
  Essa é uma prática recomendada, já que a geração de números aleatórios pode influenciar os resultados do modelo.
  Além disso, a função também configura as sementes da GPU para garantir a reprodutibilidade quando se utiliza aceleração por GPU. 
  
  Args:
      num_semente (int): número da semente a ser utilizada para inicializar as sementes das bibliotecas.
  
  References:
      http://nlp.seas.harvard.edu/2018/04/03/attention.html
      https://github.com/CyberZHG/torch-multi-head-attention/blob/master/torch_multi_head_attention/multi_head_attention.py#L15
  """
  # Define as sementes das bibliotecas random, numpy e pytorch
  random.seed(num_semente)
  np.random.seed(num_semente)
  torch.manual_seed(num_semente)
  
  # Define as sementes da GPU
  torch.backends.cudnn.deterministic = True
  torch.backends.cudnn.benchmark = False

  #torch.cuda.manual_seed(num_semente)
  #Cuda algorithms
  #torch.backends.cudnn.deterministic = True


In [34]:
hparam['num_semente'] = 13 

In [35]:
inicializa_seed(hparam['num_semente'])

## Preparando para debug e display

In [36]:
def config_display():
  """
  Esta função configura as opções de display do Pandas.
  """

  # Configurando formato saída Pandas
  # define o número máximo de colunas que serão exibidas
  pd.options.display.max_columns = None

  # define a largura máxima de uma linha
  pd.options.display.width = 1000

  # define o número máximo de linhas que serão exibidas
  pd.options.display.max_rows = 100

  # define o número máximo de caracteres por coluna
  pd.options.display.max_colwidth = 50

  # se deve exibir o número de linhas e colunas de um DataFrame.
  pd.options.display.show_dimensions = True

  # número de dígitos após a vírgula decimal a serem exibidos para floats.
  pd.options.display.precision = 7


In [37]:
def config_debug():
  """
  Esta função configura as opções de debug do PyTorch e dos pacotes
  transformers e datasets.
  """

  # Define opções de impressão de tensores para o modo científico
  torch.set_printoptions(sci_mode=True) 
  """
    Significa que valores muito grandes ou muito pequenos são mostrados em notação científica.
    Por exemplo, em vez de imprimir o número 0.0000012345 como 0.0000012345, 
    ele seria impresso como 1.2345e-06. Isso é útil em situações em que os valores dos tensores 
    envolvidos nas operações são muito grandes ou pequenos, e a notação científica permite 
    uma melhor compreensão dos números envolvidos.  
  """

  # Habilita detecção de anomalias no autograd do PyTorch
  torch.autograd.set_detect_anomaly(True)
  """
    Permite identificar operações que podem causar problemas de estabilidade numérica, 
    como gradientes explodindo ou desaparecendo. Quando essa opção é ativada, 
    o PyTorch verifica se há operações que geram valores NaN ou infinitos nos tensores 
    envolvidos no cálculo do gradiente. Se for detectado um valor anômalo, o PyTorch 
    interrompe a execução e gera uma exceção, permitindo que o erro seja corrigido 
    antes que se torne um problema maior.

    É importante notar que a detecção de anomalias pode ter um impacto significativo 
    no desempenho, especialmente em modelos grandes e complexos. Por esse motivo,
    ela deve ser usada com cautela e apenas para depuração.
  """

  # Configura variável de ambiente para habilitar a execução síncrona (bloqueante) das chamadas da API do CUDA.
  os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
  """
    o Python aguarda o término da execução de uma chamada da API do CUDA antes de executar a próxima chamada. 
    Isso é útil para depurar erros no código que envolve operações na GPU, pois permite que o erro seja capturado 
    no momento em que ocorre, e não depois de uma sequência de operações que pode tornar a origem do erro mais difícil de determinar.
    No entanto, é importante lembrar que esse modo de execução é significativamente mais lento do que a execução assíncrona, 
    que é o comportamento padrão do CUDA. Por isso, é recomendado utilizar esse comando apenas em situações de depuração 
    e removê-lo após a solução do problema.
  """

  # Define o nível de verbosity do pacote transformers para info
  # transformers.utils.logging.set_verbosity_info() 
  
  
  """
    Define o nível de detalhamento das mensagens de log geradas pela biblioteca Hugging Face Transformers 
    para o nível info. Isso significa que a biblioteca irá imprimir mensagens de log informativas sobre
    o andamento da execução, tais como tempo de execução, tamanho de batches, etc.

    Essas informações podem ser úteis para entender o que está acontecendo durante a execução da tarefa 
    e auxiliar no processo de debug. É importante notar que, em alguns casos, a quantidade de informações
    geradas pode ser muito grande, o que pode afetar o desempenho do sistema e dificultar a visualização
    das informações relevantes. Por isso, é importante ajustar o nível de detalhamento de acordo com a 
    necessidade de cada tarefa.
  
    Caso queira reduzir a quantidade de mensagens, comentar a linha acima e 
      descomentar as duas linhas abaixo, para definir o nível de verbosity como error ou warning
  
    transformers.utils.logging.set_verbosity_error()
    transformers.utils.logging.set_verbosity_warning()
  """


  # Define o modo verbose do xmode, que é utilizado no debug
  # %xmode Verbose 

  """
    Comando usado no Jupyter Notebook para controlar o modo de exibição das informações de exceções.
    O modo verbose é um modo detalhado que exibe informações adicionais ao imprimir as exceções.
    Ele inclui as informações de pilha de chamadas completa e valores de variáveis locais e globais 
    no momento da exceção. Isso pode ser útil para depurar e encontrar a causa de exceções em seu código.
    Ao usar %xmode Verbose, as informações de exceção serão impressas com mais detalhes e informações adicionais serão incluídas.

    Caso queira desabilitar o modo verbose e utilizar o modo plain, 
    comentar a linha acima e descomentar a linha abaixo:
    %xmode Plain
  """

  """
    Dica:
    1.  pdb (Python Debugger)
      Quando ocorre uma exceção em uma parte do código, o programa para a execução e exibe uma mensagem de erro 
      com informações sobre a exceção, como a linha do código em que ocorreu o erro e o tipo da exceção.

      Se você estiver depurando o código e quiser examinar o estado das variáveis ​​e executar outras operações 
      no momento em que a exceção ocorreu, pode usar o pdb (Python Debugger). Para isso, é preciso colocar o comando %debug 
      logo após ocorrer a exceção. Isso fará com que o programa pare na linha em que ocorreu a exceção e abra o pdb,
      permitindo que você explore o estado das variáveis, examine a pilha de chamadas e execute outras operações para depurar o código.


    2. ipdb
      O ipdb é um depurador interativo para o Python que oferece recursos mais avançados do que o pdb,
      incluindo a capacidade de navegar pelo código fonte enquanto depura.
      
      Você pode começar a depurar seu código inserindo o comando ipdb.set_trace() em qualquer lugar do 
      seu código onde deseja pausar a execução e começar a depurar. Quando a execução chegar nessa linha, 
      o depurador entrará em ação, permitindo que você examine o estado atual do seu programa e execute 
      comandos para investigar o comportamento.

      Durante a depuração, você pode usar comandos:
        next (para executar a próxima linha de código), 
        step (para entrar em uma função chamada na próxima linha de código) 
        continue (para continuar a execução normalmente até o próximo ponto de interrupção).

      Ao contrário do pdb, o ipdb é um depurador interativo que permite navegar pelo código fonte em que
      está trabalhando enquanto depura, permitindo que você inspecione variáveis, defina pontos de interrupção
      adicionais e até mesmo execute expressões Python no contexto do seu programa.
  """


In [38]:
config_display()

In [39]:
config_debug()

In [40]:
mostra_memoria(['cpu','gpu'])

Your runtime RAM in gb: 
 total 67.35
 available 47.86
 used 18.45
 free 19.64
 cached 27.9
 buffers 1.35
/nGPU
Mon Jun 19 18:10:54 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.116.03   Driver Version: 525.116.03   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:02:00.0 Off |                  N/A |
|100%   80C    P2   308W / 370W |  22352MiB / 24576MiB |     99%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+

## Rastro (neptune.ai)

Gerado rastro da execução no Neptune (detalhes no artigo [Rastro-DM: Mineração de Dados com Rastro](https://revista.tcu.gov.br/ojs/index.php/RTCU/article/view/1664))


### Importação de libraries para Rastro

In [41]:
import getpass, copy, tempfile, re

In [42]:
os.environ['NEPTUNE_ALLOW_SELF_SIGNED_CERTIFICATE'] = 'TRUE'
os.environ['NEPTUNE_PROJECT'] = 'marcusborela/IA386DD'

In [46]:
os.environ['NEPTUNE_API_TOKEN'] = getpass.getpass('Informe NEPTUNE_API_TOKEN')


In [44]:
tag_contexto_rastro = 'IndIR'

In [45]:
neptune_version = 0

### Código Rastro

Busca implementar o rastro proposto em [Rastro-DM: Mineração de Dados com Rastro](https://revista.tcu.gov.br/ojs/index.php/RTCU/article/view/1664), autores Marcus Vinícius Borela de Castro e Remis Balaniuk, com o apoio da [solução Neptune](https://app.neptune.ai/)



In [47]:
def converte_optimizer_state_dict(parm_optimizer)-> dict:
  """
    Recebe um objeto "parm_optimizer" que é do tipo "torch.optim.Optimizer" e retorna um dicionário 
    com informações sobre o otimizador.

    O dicionário de retorno é gerado a partir do estado do otimizador que é extraído da propriedade
    "state_dict()" do objeto "parm_optimizer", seu primeiro grupo de parâmetros do otimizador.
  """
  # return str(hparam['optimizer'])
  return parm_optimizer.state_dict()['param_groups'][0]

In [48]:
if neptune_version == 0:
  import neptune.new as neptune  
  class NeptuneRastroRun():
      se_geracao_rastro = True 
      neptune_project = ""
      tag_contexto_rastro = ""
      neptune_api_token = ""

      def __init__(self, parm_params:dict,  parm_lista_tag:list = None):
        # print(f"NeptuneRastroRun.init: se_geracao_rastro {self.__class__.se_geracao_rastro} parm_params `{parm_params} ")
        if self.__class__.se_geracao_rastro:      
          self.run_neptune = neptune.init(project=self.__class__.neptune_project, api_token=self.__class__.neptune_api_token, capture_hardware_metrics=True)
          self.run_neptune['sys/name'] = self.__class__.tag_contexto_rastro
          vparams = copy.deepcopy(parm_params)
          if "optimizer" in vparams:
            vparams["optimizer"] = converte_optimizer_state_dict(vparams["optimizer"])
          if 'criterion'  in vparams:
            vparams["criterion"] = str(vparams["criterion"])
          if 'scheduler'  in vparams:
            vparams["scheduler"] = str(type(vparams["scheduler"]))
          if 'device' in vparams:
            vparams['device'] = str(vparams["device"])
          self.device = vparams["device"]
          for tag in parm_lista_tag:
            self.run_neptune['sys/tags'].add(tag)
          self.run_neptune['parameters'] = vparams
          self.tmpDir = tempfile.mkdtemp()

      @property
      def run():
        return self.run_neptune

      @classmethod
      def ativa_geracao_rastro(cls):
        cls.se_geracao_rastro = True      

      @classmethod
      def def_contexto(cls):
        cls.se_geracao_rastro = True      

      @classmethod
      def desativa_geracao_rastro(cls):
        cls.se_geracao_rastro = False      

      @classmethod
      def retorna_status_geracao_rastro(cls):
        return cls.se_geracao_rastro      

      @classmethod
      def retorna_tag_contexto_rastro(cls):
        return cls.tag_contexto_rastro 

      @classmethod
      def inicia_contexto(cls, neptune_project, tag_contexto_rastro, neptune_api_token):
        assert '.' not in tag_contexto_rastro, "NeptuneRastroRun.init(): tag_contexto_rastro não pode possuir ponto, pois será usado para gravar nome de arquivo"      
        cls.neptune_api_token = neptune_api_token
        cls.tag_contexto_rastro = tag_contexto_rastro
        cls.neptune_project = neptune_project

      def salva_metrica(self, parm_metricas={}):
        #print(f"NeptuneRastroRun.salva_metrica: se_geracao_rastro {self.__class__.se_geracao_rastro} parm_metricas:{parm_metricas} ")
        if self.__class__.se_geracao_rastro:
          for metrica, valor in parm_metricas.items(): 
            self.run_neptune[metrica].log(valor)
  
      def gera_grafico_modelo(self, loader_train, model):
        if self.__class__.se_geracao_rastro: 
          # efetuar um forward 
          """
          se dataloader devolver x e y:
          """
          x_, y_ = next(iter(loader_train))
          x_ = x_.to(self.device)
          outputs = model(x_)
          """
          # se dataloader devolver dict:
          dados_ = next(iter(loader_train))
          outputs = model(dados_['x'].to(self.device))
          #outputs = model(x_['input_ids'].to(self.device), x_['attention_mask'].to(self.device))
          """
          nome_arquivo = os.path.join(self.tmpDir, "modelo "+ self.__class__.tag_contexto_rastro + time.strftime("%Y-%b-%d %H:%M:%S"))
          make_dot(outputs, params=dict(model.named_parameters()), show_attrs=True, show_saved=True).render(nome_arquivo, format="png")
          self.run_neptune["parameters/model_graph"].upload(nome_arquivo+'.png')
          self.run_neptune['parameters/model'] = re.sub('<bound method Module.state_dict of ', '',str(model.state_dict))      



      def stop(self):
        if self.__class__.se_geracao_rastro:         
          self.run_neptune.stop()


In [49]:
if neptune_version == 1:
  import neptune
  class NeptuneRastroRun():
      """
        Classe para geração de rastro de experimento utilizando a ferramenta Neptune.

        Busca implementar o rastro proposto em [Rastro-DM: Mineração de Dados com Rastro](https://revista.tcu.gov.br/ojs/index.php/RTCU/article/view/1664),
        autores Marcus Vinícius Borela de Castro e Remis Balaniuk, com o apoio da [solução Neptune](https://app.neptune.ai/)

        Attributes:
        -----------
        se_geracao_rastro : bool
            Indica se deve ser gerado rastro de experimento. 
        neptune_project : str
            Nome do projeto criado no Neptune. 
        tag_contexto_rastro : str
            Nome da tag utilizada para identificar o experimento.
        neptune_api_token : str
            Token utilizado para autenticação na API do Neptune. 
        run_neptune : object
            Objeto que representa o experimento no Neptune.
        device : str
            Dispositivo utilizado para o treinamento do modelo.
        tmpDir : str
          Diretório temporário utilizado para salvar gráfico do modelo.          
      """
      se_geracao_rastro = True 
      neptune_project = ""
      tag_contexto_rastro = ""
      neptune_api_token = ""

      def __init__(self, parm_params:dict,  parm_lista_tag:list = None):
        """
          Método construtor da classe NeptuneRastroRun.
          
          Args:
          - parm_params: dicionário contendo os parâmetros do modelo.
          - parm_lista_tag: lista contendo tags adicionais para o experimento.
        """      
        # print(f"NeptuneRastroRun.init: se_geracao_rastro {self.__class__.se_geracao_rastro} parm_params `{parm_params} ")
        if self.__class__.se_geracao_rastro:      
          self.run_neptune = neptune.init_run(project=self.__class__.neptune_project, api_token=self.__class__.neptune_api_token, capture_hardware_metrics=True)
          self.run_neptune['sys/name'] = self.__class__.tag_contexto_rastro
          vparams = copy.deepcopy(parm_params)
          if "optimizer" in vparams:
            vparams["optimizer"] = converte_optimizer_state_dict(vparams["optimizer"])
          if 'criterion'  in vparams:
            vparams["criterion"] = str(vparams["criterion"])
          if 'scheduler'  in vparams:
            vparams["scheduler"] = str(type(vparams["scheduler"]))
          if 'device' in vparams:
            vparams['device'] = str(vparams["device"])
          self.device = vparams["device"]
          for tag in parm_lista_tag:
            self.run_neptune['sys/tags'].add(tag)
          self.run_neptune['parameters'] = vparams
          # self.tmpDir = tempfile.mkdtemp()

      @property
      def run():
        """
        Retorna a instância do objeto run_neptune.
        """      
        return self.run_neptune

      @classmethod
      def ativa_geracao_rastro(cls):
        """
        Ativa a geração de rastro.
        """      
        cls.se_geracao_rastro = True      

      @classmethod
      def def_contexto(cls):
        """
        Define o contexto para a geração de rastro.
        """      
        cls.se_geracao_rastro = True      

      @classmethod
      def desativa_geracao_rastro(cls):
        """
        Desativa a geração de rastro.
        """      
        cls.se_geracao_rastro = False      

      @classmethod
      def retorna_status_geracao_rastro(cls):
        """
          Retorna o status da geração de rastro.
          
          Returns:
          - True se a geração de rastro está ativada, False caso contrário.
        """      
        return cls.se_geracao_rastro      

      @classmethod
      def retorna_tag_contexto_rastro(cls):
        """
          Retorna a tag do contexto de rastro.
        """      
        return cls.tag_contexto_rastro 

      @classmethod
      def inicia_contexto(cls, neptune_project, tag_contexto_rastro, neptune_api_token):
        """
        Inicia o contexto de execução no Neptune.

        Args:
            neptune_project (str): Nome do projeto no Neptune.
            tag_contexto_rastro (str): Tag que identifica o contexto de execução no Neptune.
            neptune_api_token (str): Token de acesso à API do Neptune.

        Raises:
            AssertionError: Caso a tag_contexto_rastro possua um ponto (.), 
              o que pode gerar erros na gravação de arquivo.
        """      
        assert '.' not in tag_contexto_rastro, "NeptuneRastroRun.init(): tag_contexto_rastro não pode possuir ponto, pois será usado para gravar nome de arquivo"      
        cls.neptune_api_token = neptune_api_token
        cls.tag_contexto_rastro = tag_contexto_rastro
        cls.neptune_project = neptune_project

      def salva_metrica(self, parm_metricas={}):
        """
          Salva as métricas no Neptune Run caso a geração de rastro esteja ativa.

          Parameters
          ----------
          parm_metricas: dict
              Dicionário contendo as métricas a serem salvas. As chaves devem ser os nomes das métricas e os valores devem ser
              os valores das métricas.
        """
        #print(f"NeptuneRastroRun.salva_metrica: se_geracao_rastro {self.__class__.se_geracao_rastro} parm_metricas:{parm_metricas} ")
        if self.__class__.se_geracao_rastro:
          for metrica, valor in parm_metricas.items(): 
            self.run_neptune[metrica].append(valor)
  
      def gera_grafico_modelo(self, loader_train, model):
        """
          Gera um gráfico do modelo e o envia para o Neptune. 
          Para gerar o gráfico, um forward pass é realizado em um batch de exemplos 
          de treino e o resultado é renderizado como um gráfico de nós conectados. 
          O gráfico é salvo em um arquivo .png e enviado para o Neptune como um arquivo anexo.

          Args:
              loader_train (torch.utils.data.DataLoader): DataLoader do conjunto de treinamento.
              model (torch.nn.Module): Modelo a ser visualizado.
          
          Pendente:
            Evolui para usar from io import StringIO (buffer = io.StringIO()) ao invés de tempdir 
        """    
        return

        """
        falta ajustar make_dot
        if self.__class__.se_geracao_rastro: 
          # efetuar um forward 
          batch = next(iter(loader_train))
          # falta generalizar linha abaixo. Criar função que recebe modelo e batch como parâmetro?
          outputs = model(input_ids=batch['input_ids'].to(hparam['device']), attention_mask=batch['attention_mask'].to(hparam['device']), token_type_ids=batch['token_type_ids'].to(hparam['device']), labels=batch['labels'].to(hparam['device']))
          nome_arquivo = os.path.join(self.tmpDir, "modelo "+ self.__class__.tag_contexto_rastro + time.strftime("%Y-%b-%d %H:%M:%S"))
          make_dot(outputs, params=dict(model.named_parameters()), show_attrs=True, show_saved=True).render(nome_arquivo, format="png")
          self.run_neptune["parameters/model_graph"].upload(nome_arquivo+'.png')
          self.run_neptune['parameters/model'] = re.sub('<bound method Module.state_dict of ', '',str(model.state_dict))      
        """


      def stop(self):
        """
          Para a execução do objeto Neptune. Todos os experimentos do Neptune são sincronizados com o servidor, e nenhum outro 
          experimento poderá ser adicionado a este objeto após a chamada a este método.
        """
        if self.__class__.se_geracao_rastro:         
          self.run_neptune.stop()


### Definindo parâmetros para o rastro



In [50]:
NeptuneRastroRun.inicia_contexto(os.environ['NEPTUNE_PROJECT'], tag_contexto_rastro,  os.environ['NEPTUNE_API_TOKEN'])
#NeptuneRastroRun.desativa_geracao_rastro()

# Carga dos dados

In [51]:
df = pd.read_csv(PATH_TRAIN_DATA)

In [52]:
df.shape

(284376, 4)

In [53]:
df.columns

Index(['QUERY_ID', 'DOC_ID', 'RELEVANCE', 'TYPE_RELEVANCE'], dtype='object')

In [54]:
df.head()

,QUERY_ID,DOC_ID,RELEVANCE,TYPE_RELEVANCE
0,5,1131,1,AREA
1,5,5095,0,AREA
2,5,932,0,AREA
3,5,1113,1,TEMA
4,5,1112,0,TEMA


In [56]:
df_query = pd.read_csv(PATH_QUERY)

In [59]:
df_query.shape

(16045, 11)

In [57]:
df_query.head()

,ID,TEXT,REFERENCE_LIST,PARADIGMATIC,AREA_NAME,AREA_ID_DESCRIPTOR,NORMATIVE_PROCESS_TYPE,NORMATIVE_IDENTIFICATION,NORMATIVE_DATE,NORMATIVE_AUTHOR_TYPE,NORMATIVE_AUTHOR_NAME
0,13568,É ilegal a contagem de tempo ficto de serviço ...,Lei Complementar 51/1985 || Lei Ordinária 3313...,NaN,Pessoal,1131,APOSENTADORIA,Acórdão 168/2014 - Segunda Câmara,2014-01-28,RELATOR,JOSÉ JORGE
1,11614,"SÚMULA TCU 283: Para fim de habilitação, a Adm...",NaN,SUMULA,Licitação,932,ADMINISTRATIVO,Acórdão 1613/2013 - Plenário,2013-06-26,RELATOR,JOSÉ JORGE
2,21087,A contratação de serviços por preços superiore...,NaN,NaN,Licitação,932,PRESTAÇÃO DE CONTAS,Acórdão 3936/2013 - Segunda Câmara,2013-07-09,RELATOR,ANA ARRAES
3,35016,"Não se aplica, excepcionalmente, multa aos ges...",NaN,NaN,Responsabilidade,775,REPRESENTAÇÃO,Acórdão 3231/2011 - Plenário,2011-12-07,RELATOR,AROLDO CEDRAZ
4,29370,"Em contratatações de TI, não cabe aceitar prop...",NaN,NaN,Licitação,932,REPRESENTAÇÃO,Acórdão 3231/2011 - Plenário,2011-12-07,RELATOR,AROLDO CEDRAZ


In [58]:
df_doc = pd.read_csv(PATH_DOC)

In [60]:
df_doc.shape

(13255, 28)

merge with doc

In [70]:
df = df.merge(df_doc[['ID','TEXT']], how='left', left_on='DOC_ID', right_on='ID').drop('ID',axis=1)

In [72]:
df.rename(columns={'TEXT': 'DOC_TEXT'},inplace=True)

In [73]:
df.head()

,QUERY_ID,DOC_ID,RELEVANCE,TYPE_RELEVANCE,DOC_TEXT
0,5,1131,1,AREA,"O termo é ""Pessoal"".\nPessoal tem nota de esco..."
1,5,5095,0,AREA,"O termo é ""Competência do TCU"".\nCompetência d..."
2,5,932,0,AREA,"O termo é ""Licitação"".\nLicitação tem definiçã..."
3,5,1113,1,TEMA,"O termo é ""Pensão especial"".\nPensão especial ..."
4,5,1112,0,TEMA,"O termo é ""Pensão especial de ex-combatente"".\..."


In [75]:
df = df.merge(df_query[['ID','TEXT']], how='left', left_on='QUERY_ID', right_on='ID').drop('ID',axis=1)

In [76]:
df.rename(columns={'TEXT': 'QUERY_TEXT'},inplace=True)

In [77]:
df.shape

(284376, 6)

triplas: query + id_doc + ind_relevancia (1, se positivo,  e 0 se negativo)

## Tratando qualidade dos dados

Verificando correção do arquivo!

In [78]:
print(df.isnull().sum())

QUERY_ID          0
DOC_ID            0
RELEVANCE         0
TYPE_RELEVANCE    0
DOC_TEXT          0
QUERY_TEXT        0
Length: 6, dtype: int64


In [81]:
df[['QUERY_TEXT','DOC_TEXT']].applymap(len).describe()

,QUERY_TEXT,DOC_TEXT
count,284376.0000000,284376.0000000
mean,314.2884948,1047.6093130
std,159.8254684,747.7132308
min,41.0000000,86.0000000
25%,211.0000000,606.0000000
50%,286.0000000,866.0000000
75%,383.0000000,1195.2500000
max,4212.0000000,3739.0000000


Para cada positivo, tem um negativo

In [119]:
df['RELEVANCE'].describe()

count    284376.0000000
mean          0.3333333
std           0.4714053
min           0.0000000
25%           0.0000000
50%           0.0000000
75%           1.0000000
max           1.0000000
Name: RELEVANCE, Length: 8, dtype: float64

# Dataset e Dataloader

In [85]:
class MyDataset(Dataset):
    """
      Classe para representar um dataset de texto e classes.
    """  
    def __init__(self, texts: np.ndarray, classes:list[int], tokenizer):
      """
      Inicializa um novo objeto MyDataset.

      Args:
          texts (np.ndarray): um array com as strings de texto. Cada linha deve ter 2 strings.
          classes (np.ndarray): um array com as classes de cada texto.
          tokenizer: um objeto tokenizer do Hugging Face Transformers.
          max_seq_length (int): o tamanho máximo da sequência a ser considerado.
      Raises:
          AssertionError: se os parâmetros não estiverem no formato esperado.
      """
      # Verifica se os parâmetros são do tipo esperado
      assert isinstance(texts, np.ndarray), f"Parâmetro texts deve ser do tipo np.ndarray e não {type(texts)}"
      assert texts.shape[1] == 2, "Array must have 2 columns"
      for row in texts:
          assert isinstance(row, np.ndarray) and row.shape == (2,), f"Each row in texts must have 2 elements"
          assert isinstance(row[0], str) and isinstance(row[1], str), f"Each element in texts.row must be a string e não {type(row[0])}"
      assert isinstance(classes,np.ndarray), f'classes deve ser do tipo np.ndarray e não {type(classes)}'
      assert isinstance(classes[0],np.int64), f'classes[0] deve ser do tipo numpy.int64 e não {type(classes[0])} '

      self.texts = texts
      self.classes = classes
      self.tokenizer = tokenizer
      self.max_seq_length = tokenizer.model_max_length # model.config.max_position_embeddings
      if self.max_seq_length > 64000:
        print(f"Valor de self.max_seq_length  {self.max_seq_length} indica que deve ser usado outro campo do tokenizador. Assumido 512 ")
        self.max_seq_length =  512
      # Salvar os dados dos tensores
      x_data_input_ids = []
      x_data_token_type_ids = []
      x_data_attention_masks = []
      for text_pair in tqdm(texts, desc='encoding text pair'):
          encoding = tokenizer.encode_plus(
              text_pair[0],
              text_pair[1],
              add_special_tokens=True,
              max_length=self.max_seq_length,
              padding='max_length',
              return_tensors = 'pt',
              truncation=True,
              return_attention_mask=True,
              return_token_type_ids=True
          )
          x_data_input_ids.append(encoding['input_ids'].long())
          x_data_token_type_ids.append(encoding['token_type_ids'].long())
          x_data_attention_masks.append(encoding['attention_mask'].long())
      print(F'\tVou converter lista para tensor;  Momento: {time.strftime("[%Y-%b-%d %H:%M:%S]")}')
      # squeeze: vai transformar um tensor de shape [2, 1, 322] em um tensor de shape [2, 322].

      self.x_tensor_input_ids = torch.stack(x_data_input_ids).squeeze(1)
      self.x_tensor_attention_masks = torch.stack(x_data_attention_masks).squeeze(1)
      self.x_tensor_token_type_ids = torch.stack(x_data_token_type_ids).squeeze(1)

    def __len__(self):
        """
          Retorna o tamanho do dataset (= tamanho do array texts)
        """
        return len(self.texts)
    
    def __getitem__(self, idx):
        """
          Retorna um dicionário com os dados do texto e sua classe correspondente, em um formato que pode 
          ser usado pelo dataloader do PyTorch para alimentar um modelo de aprendizado de máquina.
        """
        return {
            'input_ids': self.x_tensor_input_ids[idx],
            'attention_mask': self.x_tensor_attention_masks[idx],
            'token_type_ids': self.x_tensor_token_type_ids[idx],
            # 'labels': int(self.classes[idx])
            'labels': torch.tensor(self.classes[idx], dtype=torch.long)            
        }

In [ ]:
class MyDataset(torch.utils.data.Dataset):
    def __init__(self, mydata, seq_length, tokenizer):
        self.labels = np.array(mydata['label'])
        self.seq1 = np.array(mydata['input'])
        self.seq2 = np.array(mydata['output'])
        self.seq_length = seq_length
        self.tokenizer = tokenizer
        
    def __getitem__(self, idx):
        seq1 = self.seq1[idx]
        label = self.labels[idx]
        
        seq2 = "false"        
        if label ==1:
            seq2="true"
        

        item1= self.tokenizer(seq1, truncation=True, max_length= self.seq_length, padding='max_length')
        item2= self.tokenizer(seq2, truncation=True, max_length= 2, padding='max_length')
        
        input_ids, attention_mask = torch.tensor(item1.input_ids), torch.tensor(item1.attention_mask)
        labels = torch.tensor(item2.input_ids)     

        item={'input_ids': input_ids}
        item['attention_mask']=attention_mask
        item['label'] = labels
        
        return item

    def __len__(self):
        return len(self.labels)

In [ ]:
# adding text lenght info
# nome_modelo_ranking_pt = 'unicamp-dl/mMiniLM-L6-v2-pt-v2'
# nome_caminho_modelo_pt = "/home/borela/fontes/relevar-busca/modelo/" + nome_modelo_ranking_pt
# assert os.path.exists(nome_caminho_modelo_pt), f"Path para {nome_caminho_modelo_pt} não existe!"
# tokenizador_pt_minilm = AutoTokenizer.from_pretrained(nome_caminho_modelo_pt)


In [82]:
nome_modelo_monot5_3b = 'unicamp-dl/mt5-3B-mmarco-en-pt'
# "A mono-ptT5 reranker model (850 mb) pretrained in the BrWac corpus, finetuned for 100k steps on Portuguese translated version of MS MARCO passage dataset. The portuguese dataset was translated using Google Translate.")
nome_caminho_modelo_3b = "/home/borela/fontes/relevar-busca/modelo/" + nome_modelo_monot5_3b
assert os.path.exists(nome_caminho_modelo_3b), f"Path para {nome_caminho_modelo_3b} não existe!"
tokenizador_pt_monot5_3b = AutoTokenizer.from_pretrained(nome_caminho_modelo_3b)


/home/borela/miniconda3/envs/treinapython39/lib/python3.9/site-packages/transformers/convert_slow_tokenizer.py:446: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


#### Testando o MyDataset e o Dataloader

In [86]:
# Cria dados fictícios
texts = np.array([['This is the first query', 'text associated with query 1'],
                  ['This is the second query', 'text associated with query 2'],
                  ['This is the third query', 'text associated with query 3'],
                  ['This is the fourth query', 'text associated with query 4'],
                  ['This is the fifty query', 'text associated with query 5'],
                  ['This is the sixty query', 'text associated with query 6'],])
classes = np.ones(len(texts), dtype=int)


In [87]:
# Cria um objeto da classe MyDataset
dummy_dataset = MyDataset(texts=texts, classes=classes, tokenizer=tokenizador_pt_monot5_3b)

Valor de self.max_seq_length  1000000000000000019884624838656 indica que deve ser usado outro campo do tokenizador. Assumido 512 


encoding text pair: 100%|██████████| 6/6 [00:00<00:00, 13.03it/s]

	Vou converter lista para tensor;  Momento: [2023-Jun-19 18:42:39]


In [ ]:
MODEL_MAX_LENGTH = 512

In [91]:
# Testa o método __len__()
assert len(dummy_dataset) == 6

# Testa o método __getitem__()
sample = dummy_dataset[0]
assert set(sample.keys()) == {'input_ids', 'attention_mask', 'token_type_ids', 'labels'} # 
assert isinstance(sample['input_ids'], torch.Tensor)
assert sample['input_ids'].shape[0] == MODEL_MAX_LENGTH
assert isinstance(sample['attention_mask'], torch.Tensor)
assert sample['attention_mask'].shape[0] == MODEL_MAX_LENGTH
# assert isinstance(sample['labels'], int)
assert isinstance(sample['labels'], torch.Tensor)
print(sample['labels'].shape, sample['labels'], sample['input_ids'].shape)
print(sample['input_ids'][:20])


torch.Size([]) tensor(1) torch.Size([512])
tensor([ 1494,   339,   287,  2262,   259, 35483,     1,  7461,   259, 31929,
          514,   259, 35483,   333,     1,     0,     0,     0,     0,     0])


In [92]:
dummy_loader = DataLoader(dummy_dataset, batch_size=2, shuffle=False, num_workers= hparam['num_workers_dataloader'])

In [93]:
first_batch = next(iter(dummy_loader))

In [94]:
first_batch = BatchEncoding(first_batch).to(hparam['device'])

In [95]:
first_batch

{'input_ids': tensor([[1494,  339,  287,  ...,    0,    0,    0],
        [1494,  339,  287,  ...,    0,    0,    0]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]], device='cuda:0'), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]], device='cuda:0'), 'labels': tensor([1, 1], device='cuda:0')}

In [96]:
dummy_loader = DataLoader(dummy_dataset, batch_size=2, shuffle=False, num_workers=hparam['num_workers_dataloader'])
for ndx, epoch in tqdm(enumerate(range(1)), desc='Epochs'):
  for batch in dummy_loader:
    print(batch['labels'])
  print(f"Fim época {ndx}")


Epochs: 1it [00:00, 991.09it/s]

tensor([1, 1])
tensor([1, 1])
tensor([1, 1])
Fim época 0


In [97]:
del dummy_loader, dummy_dataset

## Experimentações e computando limites

In [126]:
tokenizador_pt_monot5_3b.pad_token_id

0

In [131]:
tokenizador_pt_monot5_3b.eos_token, tokenizador_pt_monot5_3b.eos_token_id

('</s>', 1)

In [134]:
tokenizador_pt_monot5_3b.pad_token, tokenizador_pt_monot5_3b.pad_token_id

('<pad>', 0)

In [133]:
tokenizador_pt_monot5_3b.unk_token, tokenizador_pt_monot5_3b.unk_token_id

('<unk>', 2)

In [99]:
tokenizador_pt_monot5_3b.SPECIAL_TOKENS_ATTRIBUTES

['bos_token',
 'eos_token',
 'unk_token',
 'sep_token',
 'pad_token',
 'cls_token',
 'mask_token',
 'additional_special_tokens']

In [100]:
tokenizador_pt_monot5_3b.all_special_tokens

['</s>', '<unk>', '<pad>']

In [115]:
print(tokenizador_pt_monot5_3b('</s> <unk> <pad>')['input_ids'])

[1, 259, 2, 259, 0, 1]


In [116]:
tokenizador_pt_monot5_3b.convert_ids_to_tokens(tokenizador_pt_monot5_3b('</s> <unk> <pad>')['input_ids'])

['</s>', '▁', '<unk>', '▁', '<pad>', '</s>']

In [104]:
tokenizador_pt_monot5_3b.cls_token
#tokenizador_pt_monot5_3b

Using cls_token, but it is not set yet.


In [98]:
tokenizador_pt_monot5_3b.pad_token_id,tokenizador_pt_monot5_3b.cls_token_id,tokenizador_pt_monot5_3b.sep_token_id

(0, None, None)

In [105]:
x='x, it is just a test!'
y='y, it is just a continuation!'

In [106]:
print(tokenizador_pt_monot5_3b(x,y)['input_ids'])

[259, 329, 261, 609, 339, 1627, 259, 262, 2978, 309, 1, 259, 276, 261, 609, 339, 1627, 259, 262, 10083, 1300, 309, 1]


In [107]:
print(tokenizador_pt_monot5_3b.encode_plus(x,y)['input_ids'])

[259, 329, 261, 609, 339, 1627, 259, 262, 2978, 309, 1, 259, 276, 261, 609, 339, 1627, 259, 262, 10083, 1300, 309, 1]


In [110]:
# Converter os input_ids para os vocábulos correspondentes
vocabularies = tokenizador_pt_monot5_3b.convert_ids_to_tokens(tokenizador_pt_monot5_3b.encode_plus(x,y)['input_ids'])
vocabularies2 = tokenizador_pt_monot5_3b.convert_ids_to_tokens(tokenizador_pt_monot5_3b(x,y)['input_ids'])
vocabularies == vocabularies2

True

In [111]:
# Limpa o cache da memória da GPU
torch.cuda.empty_cache()

In [112]:
gc.collect()

2087

# Divisão em treino e validação (de forma estratificada)

In [113]:
from sklearn.model_selection import train_test_split

In [117]:
hparam['percent_test_size'] = .05 #@param [.10, .15, .20] {type:'raw'}

O dataset de validação será sempre no leiaute qp (para permitir comparar)

In [120]:
X_train, X_valid, Y_train, Y_valid = train_test_split(df[['QUERY_TEXT','DOC_TEXT']].values, 
                                                      df['RELEVANCE'].values,
                                                      test_size=hparam['percent_test_size'], 
                                                      stratify=df['RELEVANCE'].values, random_state=hparam['num_semente'])

In [121]:
X_train.shape

(270157, 2)

In [122]:
X_train.shape, type(X_valid), type(Y_valid), X_valid.shape, Y_valid.shape

((270157, 2), numpy.ndarray, numpy.ndarray, (14219, 2), (14219,))

Para cada relevante, tem 2 não relevantes

In [123]:
print(np.unique(Y_train, return_counts=True), '\n', np.unique(Y_valid, return_counts=True))

(array([0, 1]), array([180105,  90052])) 
 (array([0, 1]), array([9479, 4740]))


In [124]:
X_train[0]

array(['A sentença proferida pelo juízo cível, sob qualquer fundamento, não vincula a decisão proferida pelo TCU. Apenas a sentença absolutória no juízo criminal fundada no reconhecimento da inexistência material do fato ou na negativa de autoria tem habilidade para impedir a responsabilização civil e administrativa do agente. ',
       'O termo é "Princípio da independência das instâncias".\nPrincípio da independência das instâncias tem definição: "Significa que a esfera penal, civil e administrativa, são totalmente independentes. A matéria decidida em uma esfera, não vinculará o que será decidido nas demais.".\nPrincípio da independência das instâncias tem nota de escopo: "A matéria decidida na esfera penal vinculará as demais esferas (civil e administrativa). Isso é dizer que, coisa julgada na esfera penal faz coisa julgada no cível e/ou administrativo, mas, inversamente ocorre que, a coisa julgada na esfera cível e/ou administrativa, não faz coisa julgada na esfera penal.".\nPrincí

In [125]:
Y_train[:4]

array([1, 0, 1, 0])

# Treinamento do ranqueador

Fonte: https://colab.research.google.com/drive/10etP7Lb915EC-uEuf1IKC8DYkyg_om6-?usp=sharing

In this notebook, we will finetuned and evaluate a BERT-based model on a document classification task.

Our dataset will be a smaller version of the IMDB Sentiment Analysis dataset.

To finetune and evaluate faster, we will be using [miniLM](https://huggingface.co/microsoft/MiniLM-L12-H384-uncased), which is small BERT-like model distilled from a larger one.

## Carga do modelo para finetuning (pré-treinado no MSMARCO)

"Começar treino do miniLM já treinado no MS MARCO":  cross-encoder/ms-marco-MiniLM-L-6-v2

In [49]:
# hparam['model_name'] = 'cross-encoder/ms-marco-MiniLM-L-6-v2'
hparam['model_name'] = 'microsoft/MiniLM-L12-H384-uncased'
model = AutoModelForSequenceClassification.from_pretrained(hparam['model_name'], num_labels=2).to(hparam['device'])
tokenizer = AutoTokenizer.from_pretrained(hparam['model_name'])


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at microsoft/MiniLM-L12-H384-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [50]:
tokenizer = AutoTokenizer.from_pretrained(hparam['model_name'])

### Experimentações e computando limites

In [179]:
tokenizer.pad_token_id,tokenizer.cls_token_id,tokenizer.sep_token_id

(0, 101, 102)

In [180]:
x='x, it is just a test!'
y='y, it is just a continuation!'

In [181]:
print(tokenizer(x,y)['input_ids'])

[101, 1060, 1010, 2009, 2003, 2074, 1037, 3231, 999, 102, 1061, 1010, 2009, 2003, 2074, 1037, 13633, 999, 102]


In [182]:
print(tokenizer.encode_plus(x,y)['input_ids'])

[101, 1060, 1010, 2009, 2003, 2074, 1037, 3231, 999, 102, 1061, 1010, 2009, 2003, 2074, 1037, 13633, 999, 102]


In [183]:
model.config.max_position_embeddings

512

In [184]:
tokenizer.vocab_size

30522

## Definindo parâmetros e informações para o rastro (hparam) - sobre dados

In [185]:
X_train.shape[0] , X_valid.shape[0]

(82298, 4332)

## Carregando o Dataset e os Dataloaders


In [51]:
datasets_carregados_previamente = True #@param {type:"boolean"}
sufixo_treino = '30k_ds_negative_full' #  '10k_ds'
print(datasets_carregados_previamente)

True


In [52]:
if datasets_carregados_previamente:
  with open(f"{DIRETORIO_TREINO}/dataset_train_{sufixo_treino}.pt",'rb') as f:
    buffer = io.BytesIO(f.read())
  dataset_train = torch.load(buffer)
  with open(f"{DIRETORIO_TREINO}/dataset_valid_{sufixo_treino}.pt",'rb') as f:
    buffer = io.BytesIO(f.read())
  dataset_valid = torch.load(buffer)
  print('Datasets carregados')   

Datasets carregados


In [188]:
%%time
if not datasets_carregados_previamente:
    print("carregando valid_dataset")  
    dataset_valid = MyDataset(texts=X_valid, classes=Y_valid, tokenizer=tokenizer)
    torch.save(dataset_valid, f"{DIRETORIO_TREINO}/dataset_valid_{sufixo_treino}.pt")


carregando valid_dataset
Valor de self.max_seq_length  1000000000000000019884624838656 indica que deve ser usado outro campo do tokenizador. Assumido 512 


encoding text pair: 100%|██████████| 4332/4332 [00:05<00:00, 819.90it/s]


	Vou converter lista para tensor;  Momento: [2023-May-03 18:10:01]
CPU times: user 5.37 s, sys: 28.1 ms, total: 5.39 s
Wall time: 5.38 s


In [189]:
%%time
if not datasets_carregados_previamente:
    print("carregando train_dataset")  
    dataset_train = MyDataset(texts=X_train, classes=Y_train, tokenizer=tokenizer)
    torch.save(dataset_train, f"{DIRETORIO_TREINO}/dataset_train_{sufixo_treino}.pt")

carregando train_dataset
Valor de self.max_seq_length  1000000000000000019884624838656 indica que deve ser usado outro campo do tokenizador. Assumido 512 


encoding text pair: 100%|██████████| 82298/82298 [01:53<00:00, 727.77it/s]


	Vou converter lista para tensor;  Momento: [2023-May-03 18:11:55]
CPU times: user 1min 54s, sys: 701 ms, total: 1min 55s
Wall time: 1min 54s


In [53]:
len(dataset_train), len(dataset_valid)

(82298, 4332)

In [54]:
# visto que "max_position_embeddings": 512 em https://huggingface.co/microsoft/MiniLM-L12-H384-uncased/blob/main/config.json
hparam['vocab_size'] = tokenizer.vocab_size
hparam['batch_size'] = 32
hparam['num_epochs'] = 8  
hparam['num_sentenca_train'] = len(dataset_train) # X_train.shape[0] # ou 37400
hparam['num_sentenca_valid'] = len(dataset_valid) # X_valid.shape[0] # 


In [55]:
dataloader_train = DataLoader(dataset_train, batch_size=hparam['batch_size'], shuffle=True)
dataloader_valid = DataLoader(dataset_valid, batch_size=hparam['batch_size'], shuffle=False)

## Funções auxiliares de treinamento

In [69]:
def treina_modelo (model, hparam, parm_dataloader_train, parm_dataloader_valid, parm_se_salva_modelo:bool=True, parm_se_gera_rastro:bool=True):
    global DIRETORIO_TREINO, tag_contexto_rastro

    if parm_se_gera_rastro:
        rastro_neptune = NeptuneRastroRun(hparam, parm_lista_tag= tag_contexto_rastro)

    history = []
    n_examples = 0
    time_inicio_treino = time.time()

    try:
        best_metrica_validacao = 0 # float('-inf')
        best_epoch = 0
        

        for epoch in tqdm(range(hparam['num_epochs']), desc='Epochs'):
            model.train()

            train_losses = []
            correct = 0
            for batch in tqdm(parm_dataloader_train, mininterval=0.5, desc='Train', disable=False):
                #outputs = model(input_ids=batch['input_ids'].to(hparam['device']),
                #                attention_mask=batch['attention_mask'].to(hparam['device']), 
                #                token_type_ids=batch['token_type_ids'].to(hparam['device']), 
                #                labels=batch['labels'].to(hparam['device']))
                hparam['optimizer'].zero_grad()
                outputs = model(**BatchEncoding(batch).to(hparam['device']))
                loss = outputs.loss
                loss.backward()
                hparam['optimizer'].step()
                if 'scheduler' in hparam:
                    hparam['scheduler'].step()
                n_examples += len(batch['input_ids'])  # Increment of batch size
                train_losses.append(loss.cpu().item())

                preds = outputs.logits.argmax(dim=1)
                correct += (preds == batch['labels'].to(hparam['device'])).sum().item()    

            if isinstance(parm_dataloader_train,torch.utils.data.dataloader.DataLoader): 
                accuracy_train = correct / len(parm_dataloader_train.dataset)
            elif isinstance(parm_dataloader_train,list):  # usado para overfit em um batch
                accuracy_train = correct / len(parm_dataloader_train[0]['labels'])    
                assert len(parm_dataloader_train) == 1, f"Calculo de accuracy para overfit em um batch precisa ser revisado"

            loss_train_mean = mean(train_losses)

            accuracy_valid, mean_loss_valid = evaluate(model=model, dataloader=parm_dataloader_valid, set_name='Valid')

            
            print(f'Epoch: {epoch + 1} Training loss: {loss_train_mean:0.2f} accuracy: {accuracy_train:0.3f} Em validação: loss: {mean_loss_valid:0.3f}; accuracy: {accuracy_valid:0.3f}')
            metrica_rastro = {"train/loss": loss_train_mean, 
                            "train/accuracy": accuracy_train, 
                            "train/n_examples": n_examples, 
                            "train/learning_rate": hparam["optimizer"].param_groups[0]["lr"],
                            "valid/loss": mean_loss_valid,
                            "valid/accuracy": accuracy_valid}  
            history.append(metrica_rastro)
            if parm_se_gera_rastro:   
                rastro_neptune.salva_metrica(metrica_rastro)

            path_modelo = f'{DIRETORIO_TREINO}/model_fim_treino_epoca_{epoch}.pt'
            # torch.save(model, path_modelo) 
            model.save_pretrained(path_modelo)   
            print(f"Modelo salvo para época {epoch} salvo em {path_modelo}")

            if accuracy_valid > best_metrica_validacao:
                best_model_dict = model.state_dict()
                best_metrica_validacao = accuracy_valid
                best_epoch = epoch + 1
                # print(f'encontrado best model Epoch: {epoch + 1} ')
            elif hparam['early_stop'] < (epoch + 1 - best_epoch):
                print(f"Parando por critério de early_stop na época {epoch + 1} sendo best_epoch {best_epoch} e ealy_stop {hparam['early_stop']}")
                break # for epoch


        if parm_se_salva_modelo:
            model.load_state_dict(best_model_dict)
            model.to(hparam['device'])
            path_modelo = f'{DIRETORIO_TREINO}/model_fim_treino_{time.strftime("%Y-%b-%d %H:%M:%S")}.pt'
            # torch.save(model, path_modelo) 
            model.save_pretrained(path_modelo)   
            print(f"Modelo com melhor resultado em validação (epoch {best_epoch}) salvo após treino em {path_modelo}")

        # calculando tempo gasto e médio por 
        tempo_treino = time.time() - time_inicio_treino   
        if parm_se_gera_rastro:
            rastro_neptune.run_neptune["context/tempo_treino"] = tempo_treino
            rastro_neptune.run_neptune["context/best_epoch"] = best_epoch
            rastro_neptune.run_neptune["context/tempo_treino_epoc_mean"] = tempo_treino/hparam['num_epochs']
            # rastro_neptune.gera_grafico_modelo(parm_dataloader_train, model)  

    finally: # para não deixar em aberto a execução no Neptune
        if parm_se_gera_rastro:
            rastro_neptune.stop()
  
    return {"loss_validacao":mean_loss_valid, "loss_treino":loss_train_mean, "best_metrica_validacao":best_metrica_validacao,  "best_epoch": best_epoch, "history": history}


In [70]:
def count_parameters(model):
    """
    Conta o número de parâmetros treináveis em um modelo PyTorch.

    Args:
        model (torch.nn.Module): O modelo PyTorch a ser avaliado.

    Returns:
        int: O número de parâmetros treináveis do modelo.

    """
    # Retorna a soma do número de elementos em cada tensor de parâmetro que requer gradiente.
    # A propriedade "requires_grad" é definida como True para todos os tensores de parâmetro que 
    # precisam ser treinados, enquanto que para aqueles que não precisam, ela é definida como False.
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [71]:
# We first define the evaluation function to measure accuracy and loss
def evaluate(model, dataloader, set_name):
    losses = []
    correct = 0
    model.eval()
    with torch.no_grad():
        for ndx, batch in tqdm(enumerate(dataloader), mininterval=0.5, desc=set_name, disable=False):
            #input_ids = batch['input_ids'].to(hparam['device'])
            #token_type_ids = batch['token_type_ids'].to(hparam['device'])
            #attention_mask = batch['attention_mask'].to(hparam['device'])                                              
            #labels = batch['labels'].to(hparam['device'])                                              
            #outputs = model(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, labels=labels)
            outputs = model(**BatchEncoding(batch).to(hparam['device']))
            loss_val = outputs.loss
            losses.append(loss_val.cpu().item())
            preds = outputs.logits.argmax(dim=1)
            correct += (preds == batch['labels'].to(hparam['device'])).sum().item()
    if isinstance(dataloader,torch.utils.data.dataloader.DataLoader): 
        accuracy = correct / len(dataloader.dataset)
    elif isinstance(dataloader,list):  # usado para overfit em um batch
        accuracy = correct / len(dataloader[0]['labels'])
        assert len(dataloader) == 1, f"Calculo de accuracy para overfit em um batch precisa ser revisado"
    mean_loss = mean(losses)
    return accuracy, mean_loss

In [59]:
from torch import nn
from torch import optim
from tqdm.auto import tqdm
from transformers import get_linear_schedule_with_warmup

In [60]:
def inicializa_modelo():
  global model, hparam

  model = AutoModelForSequenceClassification.from_pretrained(hparam['model_name'], num_labels=2).to(hparam['device'])
  hparam['learning_rate']=1e-3 # 3e-5 # 1e-3
  hparam['num_params'] = count_parameters(model)
  # print(f"Number of model parameters: {hparam['num_params']}")
  hparam['early_stop'] = 5
  hparam['optimizer'] = torch.optim.SGD(model.parameters(), lr=hparam['learning_rate'])  
  # hparam['num_training_steps'] = hparam['num_epochs'] * len(dataloader_train)
  # hparam['num_warmup_steps'] = int(hparam['num_training_steps'] * 0.1)
  # hparam['weight_decay'] = 1e-4
  # hparam['amsgrad']=False
  # hparam['optimizer'] = torch.optim.Adam(model.parameters(), lr=hparam['learning_rate'], weight_decay= hparam['weight_decay'], amsgrad=hparam['amsgrad'])
  # hparam['scheduler'] = get_linear_schedule_with_warmup(hparam['optimizer'], hparam['num_warmup_steps'], hparam['num_training_steps'])  



## Evaluating on the valid dataset before training

In [201]:
accuracy, mean_loss = evaluate(model=model, dataloader=dataloader_valid, set_name='Valid')
print(f'Em validação: loss: {mean_loss:0.3f}; accuracy: {accuracy:0.3f}')

Valid: 68it [00:14,  4.57it/s]

Em validação: loss: 0.693; accuracy: 0.500


## Testando em poucos dados: um batch apenas. Esperado overfit loss próxima de zero.

Por algum motivo a investigar quando passo dataset como lista não está gerando aprendizado. Mas quando passo normal, o modelo está aprendendo. 




In [62]:
hparam['num_epochs'] = 5
hparam['early_stop'] = 1

In [63]:
inicializa_modelo()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at microsoft/MiniLM-L12-H384-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [64]:
hparam

{'num_workers_dataloader': 0,
 'device': device(type='cuda', index=0),
 'num_semente': 13,
 'model_name': 'microsoft/MiniLM-L12-H384-uncased',
 'vocab_size': 30522,
 'batch_size': 32,
 'num_epochs': 5,
 'num_sentenca_train': 82298,
 'num_sentenca_valid': 4332,
 'early_stop': 5,
 'learning_rate': 0.001,
 'num_params': 33360770,
 'optimizer': SGD (
 Parameter Group 0
     dampening: 0
     differentiable: False
     foreach: None
     lr: 0.001
     maximize: False
     momentum: 0
     nesterov: False
     weight_decay: 0
 )}

In [65]:
resultado =  treina_modelo (model, hparam,  [next(iter(dataloader_train))], [next(iter(dataloader_valid))], parm_se_salva_modelo=False, parm_se_gera_rastro=False)

Train: 100%|██████████| 1/1 [00:01<00:00,  1.26s/it]
Valid: 1it [00:00,  8.80it/s]
Epochs:  20%|██        | 1/5 [00:01<00:05,  1.38s/it]

Epoch: 1 Training loss: 0.70 accuracy: 0.375 Em validação: loss: 0.698; accuracy: 0.438


Train: 100%|██████████| 1/1 [00:00<00:00,  1.73it/s]
Valid: 1it [00:00,  8.78it/s]
Epochs:  40%|████      | 2/5 [00:02<00:02,  1.02it/s]

Epoch: 2 Training loss: 0.70 accuracy: 0.375 Em validação: loss: 0.697; accuracy: 0.438


Train: 100%|██████████| 1/1 [00:00<00:00,  1.74it/s]
Valid: 1it [00:00,  8.78it/s]
Epochs:  60%|██████    | 3/5 [00:02<00:01,  1.18it/s]

Epoch: 3 Training loss: 0.70 accuracy: 0.375 Em validação: loss: 0.697; accuracy: 0.438


Train: 100%|██████████| 1/1 [00:00<00:00,  1.75it/s]
Valid: 1it [00:00,  8.74it/s]
Epochs:  80%|████████  | 4/5 [00:03<00:00,  1.27it/s]

Epoch: 4 Training loss: 0.70 accuracy: 0.375 Em validação: loss: 0.697; accuracy: 0.438


Train: 100%|██████████| 1/1 [00:00<00:00,  1.75it/s]
Valid: 1it [00:00,  8.78it/s]
Epochs: 100%|██████████| 5/5 [00:04<00:00,  1.20it/s]

Epoch: 5 Training loss: 0.70 accuracy: 0.375 Em validação: loss: 0.696; accuracy: 0.438


## Treino do conjunto completo  10k + dataset_colegas

In [75]:
inicializa_modelo()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at microsoft/MiniLM-L12-H384-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
hparam['num_epochs'] = 15
hparam['early_stop'] = 3

In [76]:
hparam

{'num_workers_dataloader': 0,
 'device': device(type='cuda', index=0),
 'num_semente': 13,
 'model_name': 'microsoft/MiniLM-L12-H384-uncased',
 'vocab_size': 30522,
 'batch_size': 32,
 'num_epochs': 15,
 'num_sentenca_train': 48792,
 'num_sentenca_valid': 48792,
 'early_stop': 5,
 'learning_rate': 0.001,
 'num_params': 33360770,
 'amsgrad': False,
 'weight_decay': 0.0001,
 'num_training_steps': 22875,
 'num_warmup_steps': 2287,
 'optimizer': SGD (
 Parameter Group 0
     dampening: 0
     differentiable: False
     foreach: None
     lr: 0.001
     maximize: False
     momentum: 0
     nesterov: False
     weight_decay: 0
 ),
 'scheduler': <torch.optim.lr_scheduler.LambdaLR at 0x7f317845a2e0>}

In [77]:
resultado = treina_modelo(model, hparam,  dataloader_train, dataloader_valid)

/tmp/ipykernel_245210/1999415923.py:12: NeptuneDeprecationWarning: `init` is deprecated, use `init_run` instead. We'll end support of it in `neptune-client==1.0.0`. For details, see https://docs.neptune.ai/setup/neptune-client_1-0_release_changes
  self.run_neptune = neptune.init(project=self.__class__.neptune_project, api_token=self.__class__.neptune_api_token, capture_hardware_metrics=True)


https://app.neptune.ai/marcusborela/IA386DD/e/IAD-84
Remember to stop your run once you’ve finished logging your metadata (https://docs.neptune.ai/api/run#stop). It will be stopped automatically only when the notebook kernel/interactive console is terminated.


/tmp/ipykernel_245210/1999415923.py:26: NeptuneDeprecationWarning: The object you're logging will be implicitly cast to a string. We'll end support of this behavior in `neptune-client==1.0.0`. To log the object as a string, use `str(object)` or `repr(object)` instead. For details, see https://docs.neptune.ai/setup/neptune-client_1-0_release_changes
  self.run_neptune['parameters'] = vparams
Train: 100%|██████████| 1525/1525 [14:52<00:00,  1.71it/s]
Valid: 1525it [03:09,  8.04it/s]
Epochs:   7%|▋         | 1/15 [18:02<4:12:32, 1082.31s/it]

Epoch: 1 Training loss: 0.67 accuracy: 0.600 Em validação: loss: 0.673; accuracy: 0.600


Train: 100%|██████████| 1525/1525 [14:57<00:00,  1.70it/s]
Valid: 1525it [03:09,  8.04it/s]
Epochs:  13%|█▎        | 2/15 [36:09<3:55:10, 1085.45s/it]

...

Train: 100%|██████████| 1525/1525 [14:58<00:00,  1.70it/s]
Valid: 1525it [03:13,  7.88it/s]
Epochs:  93%|█████████▎| 14/15 [4:14:09<18:09, 1089.48s/it]

Epoch: 14 Training loss: 0.00 accuracy: 1.000 Em validação: loss: 0.001; accuracy: 1.000


Train: 100%|██████████| 1525/1525 [14:57<00:00,  1.70it/s]
Valid: 1525it [03:07,  8.14it/s]
Epochs: 100%|██████████| 15/15 [4:32:13<00:00, 1088.93s/it]

Epoch: 15 Training loss: 0.00 accuracy: 1.000 Em validação: loss: 0.001; accuracy: 1.000
Modelo com melhor resultado em validação (epoch 15) salvo após treino em /home/borela/fontes/deep_learning_em_buscas_unicamp/local/inpars/treino/model_fim_treino_2023-May-03 04:50:18.pt
Shutting down background jobs, please wait a moment...


Done!
Waiting for the remaining 27 operations to synchronize with Neptune. Do not kill this process.
All 27 operations synced, thanks for waiting!
Explore the metadata in the Neptune app:
https://app.neptune.ai/marcusborela/IA386DD/e/IAD-84


## Treino do conjunto completo  30k + dataset_colegas + msmarco_tiny

In [ ]:
inicializa_modelo()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at microsoft/MiniLM-L12-H384-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
hparam['num_epochs'] = 15
hparam['early_stop'] = 2

In [ ]:
hparam

{'num_workers_dataloader': 0,
 'device': device(type='cuda', index=0),
 'num_semente': 13,
 'model_name': 'microsoft/MiniLM-L12-H384-uncased',
 'vocab_size': 30522,
 'batch_size': 32,
 'num_epochs': 15,
 'num_sentenca_train': 106023,
 'num_sentenca_valid': 3280,
 'learning_rate': 0.001,
 'num_params': 33360770,
 'early_stop': 2,
 'optimizer': SGD (
 Parameter Group 0
     dampening: 0
     differentiable: False
     foreach: None
     lr: 0.001
     maximize: False
     momentum: 0
     nesterov: False
     weight_decay: 0
 )}

In [ ]:
# Limpa o cache da memória da GPU
torch.cuda.empty_cache()

In [ ]:
resultado = treina_modelo(model, hparam,  dataloader_train, dataloader_valid)

/tmp/ipykernel_274689/1999415923.py:12: NeptuneDeprecationWarning: `init` is deprecated, use `init_run` instead. We'll end support of it in `neptune-client==1.0.0`. For details, see https://docs.neptune.ai/setup/neptune-client_1-0_release_changes
  self.run_neptune = neptune.init(project=self.__class__.neptune_project, api_token=self.__class__.neptune_api_token, capture_hardware_metrics=True)


https://app.neptune.ai/marcusborela/IA386DD/e/IAD-86
Remember to stop your run once you’ve finished logging your metadata (https://docs.neptune.ai/api/run#stop). It will be stopped automatically only when the notebook kernel/interactive console is terminated.


/tmp/ipykernel_274689/1999415923.py:26: NeptuneDeprecationWarning: The object you're logging will be implicitly cast to a string. We'll end support of this behavior in `neptune-client==1.0.0`. To log the object as a string, use `str(object)` or `repr(object)` instead. For details, see https://docs.neptune.ai/setup/neptune-client_1-0_release_changes
  self.run_neptune['parameters'] = vparams
Train: 100%|██████████| 3314/3314 [32:50<00:00,  1.68it/s]
Valid: 103it [00:12,  8.04it/s]
Epochs:   7%|▋         | 1/15 [33:03<7:42:52, 1983.74s/it]

Epoch: 1 Training loss: 0.69 accuracy: 0.531 Em validação: loss: 0.690; accuracy: 0.532


Train: 100%|██████████| 3314/3314 [32:40<00:00,  1.69it/s]
Valid: 103it [00:12,  7.93it/s]
Epochs:  13%|█▎        | 2/15 [1:05:56<7:08:27, 1977.51s/it]

...

Train: 100%|██████████| 3314/3314 [33:03<00:00,  1.67it/s]
Valid: 103it [00:12,  8.02it/s]
Epochs:  60%|██████    | 9/15 [4:56:54<3:18:43, 1987.31s/it]

Epoch: 9 Training loss: 0.00 accuracy: 0.999 Em validação: loss: 0.008; accuracy: 0.998


Train: 100%|██████████| 3314/3314 [33:28<00:00,  1.65it/s]
Valid: 103it [00:12,  7.99it/s]
Epochs:  60%|██████    | 9/15 [5:30:36<3:40:24, 2204.02s/it]

Epoch: 10 Training loss: 0.00 accuracy: 0.999 Em validação: loss: 0.007; accuracy: 0.999
Parando por critério de early_stop na época 10 sendo best_epoch 7 e ealy_stop 2
Modelo com melhor resultado em validação (epoch 7) salvo após treino em /home/borela/fontes/deep_learning_em_buscas_unicamp/local/inpars/treino/model_fim_treino_2023-May-03 16:05:27.pt
Shutting down background jobs, please wait a moment...


Done!
Waiting for the remaining 22 operations to synchronize with Neptune. Do not kill this process.
All 22 operations synced, thanks for waiting!
Explore the metadata in the Neptune app:
https://app.neptune.ai/marcusborela/IA386DD/e/IAD-86


In [ ]:
resultado

{'loss_validacao': 0.0065397137687858015,
 'loss_treino': 0.0035025183040803686,
 'best_metrica_validacao': 0.998780487804878,
 'best_epoch': 7,
 'history': [{'train/loss': 0.69064182661784,
   'train/accuracy': 0.5306678739518783,
   'train/n_examples': 106023,
   'train/learning_rate': 0.001,
   'valid/loss': 0.6898829393016482,
   'valid/accuracy': 0.5317073170731708},
  {'train/loss': 0.6887287442124458,
   'train/accuracy': 0.5316865208492497,
   'train/n_examples': 212046,
   'train/learning_rate': 0.001,
   'valid/loss': 0.686330312664069,
   'valid/accuracy': 0.5317073170731708},
  {'train/loss': 0.6586694749601648,
   'train/accuracy': 0.5954368391764051,
   'train/n_examples': 318069,
   'train/learning_rate': 0.001,
   'valid/loss': 0.539984837897773,
   'valid/accuracy': 0.7289634146341464},
  {'train/loss': 0.062497507569177206,
   'train/accuracy': 0.9828622091432991,
   'train/n_examples': 424092,
   'train/learning_rate': 0.001,
   'valid/loss': 0.013665476160630295,
  

## Treino do conjunto completo  30k + dataset_colegas

Salvando um modelo a cada época

In [72]:
inicializa_modelo()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at microsoft/MiniLM-L12-H384-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [73]:
hparam['num_epochs'] = 15
hparam['early_stop'] = 2

In [74]:
hparam

{'num_workers_dataloader': 0,
 'device': device(type='cuda', index=0),
 'num_semente': 13,
 'model_name': 'microsoft/MiniLM-L12-H384-uncased',
 'vocab_size': 30522,
 'batch_size': 32,
 'num_epochs': 15,
 'num_sentenca_train': 82298,
 'num_sentenca_valid': 4332,
 'early_stop': 2,
 'learning_rate': 0.001,
 'num_params': 33360770,
 'optimizer': SGD (
 Parameter Group 0
     dampening: 0
     differentiable: False
     foreach: None
     lr: 0.001
     maximize: False
     momentum: 0
     nesterov: False
     weight_decay: 0
 )}

In [75]:
# Limpa o cache da memória da GPU
torch.cuda.empty_cache()

In [76]:
resultado = treina_modelo(model, hparam,  dataloader_train, dataloader_valid)

/tmp/ipykernel_299198/1999415923.py:12: NeptuneDeprecationWarning: `init` is deprecated, use `init_run` instead. We'll end support of it in `neptune-client==1.0.0`. For details, see https://docs.neptune.ai/setup/neptune-client_1-0_release_changes
  self.run_neptune = neptune.init(project=self.__class__.neptune_project, api_token=self.__class__.neptune_api_token, capture_hardware_metrics=True)


https://app.neptune.ai/marcusborela/IA386DD/e/IAD-87
Remember to stop your run once you’ve finished logging your metadata (https://docs.neptune.ai/api/run#stop). It will be stopped automatically only when the notebook kernel/interactive console is terminated.


/tmp/ipykernel_299198/1999415923.py:26: NeptuneDeprecationWarning: The object you're logging will be implicitly cast to a string. We'll end support of this behavior in `neptune-client==1.0.0`. To log the object as a string, use `str(object)` or `repr(object)` instead. For details, see https://docs.neptune.ai/setup/neptune-client_1-0_release_changes
  self.run_neptune['parameters'] = vparams
Train: 100%|██████████| 2572/2572 [33:03<00:00,  1.30it/s]
Valid: 136it [00:16,  8.14it/s]
Epochs:   7%|▋         | 1/15 [33:19<7:46:39, 1999.94s/it]

Epoch: 1 Training loss: 0.69 accuracy: 0.507 Em validação: loss: 0.693; accuracy: 0.500
Modelo salvo para época 0 salvo em /home/borela/fontes/deep_learning_em_buscas_unicamp/local/inpars/treino/model_fim_treino_epoca_0.pt


Train: 100%|██████████| 2572/2572 [29:14<00:00,  1.47it/s]
Valid: 136it [00:17,  7.99it/s]
Epochs:  13%|█▎        | 2/15 [1:02:51<6:44:16, 1865.85s/it]

...

Epoch: 11 Training loss: 0.01 accuracy: 0.997 Em validação: loss: 0.015; accuracy: 0.996
Modelo salvo para época 10 salvo em /home/borela/fontes/deep_learning_em_buscas_unicamp/local/inpars/treino/model_fim_treino_epoca_10.pt


Train: 100%|██████████| 2572/2572 [25:12<00:00,  1.70it/s]
Valid: 136it [00:16,  8.01it/s]
Epochs:  80%|████████  | 12/15 [5:47:37<1:21:01, 1620.34s/it]

Epoch: 12 Training loss: 0.01 accuracy: 0.998 Em validação: loss: 0.009; accuracy: 0.997
Modelo salvo para época 11 salvo em /home/borela/fontes/deep_learning_em_buscas_unicamp/local/inpars/treino/model_fim_treino_epoca_11.pt


Train: 100%|██████████| 2572/2572 [25:11<00:00,  1.70it/s]
Valid: 136it [00:16,  8.06it/s]
Epochs:  80%|████████  | 12/15 [6:13:06<1:33:16, 1865.51s/it]


Epoch: 13 Training loss: 0.01 accuracy: 0.998 Em validação: loss: 0.011; accuracy: 0.997
Modelo salvo para época 12 salvo em /home/borela/fontes/deep_learning_em_buscas_unicamp/local/inpars/treino/model_fim_treino_epoca_12.pt
Parando por critério de early_stop na época 13 sendo best_epoch 10 e ealy_stop 2
Modelo com melhor resultado em validação (epoch 10) salvo após treino em /home/borela/fontes/deep_learning_em_buscas_unicamp/local/inpars/treino/model_fim_treino_2023-May-04 00:43:11.pt
Shutting down background jobs, please wait a moment...
Done!
Waiting for the remaining 49 operations to synchronize with Neptune. Do not kill this process.
All 49 operations synced, thanks for waiting!
Explore the metadata in the Neptune app:
https://app.neptune.ai/marcusborela/IA386DD/e/IAD-87


In [77]:
resultado

{'loss_validacao': 0.010857868868376553,
 'loss_treino': 0.007026257659202698,
 'best_metrica_validacao': 0.997229916897507,
 'best_epoch': 10,
 'history': [{'train/loss': 0.6930233154150413,
   'train/accuracy': 0.5073634839242752,
   'train/n_examples': 82298,
   'train/learning_rate': 0.001,
   'valid/loss': 0.692666282548624,
   'valid/accuracy': 0.5},
  {'train/loss': 0.6924362652131224,
   'train/accuracy': 0.5251646455563926,
   'train/n_examples': 164596,
   'train/learning_rate': 0.001,
   'valid/loss': 0.6919465761850861,
   'valid/accuracy': 0.5136195752539243},
  {'train/loss': 0.6913582730348893,
   'train/accuracy': 0.546040000972077,
   'train/n_examples': 246894,
   'train/learning_rate': 0.001,
   'valid/loss': 0.6905831322073936,
   'valid/accuracy': 0.5152354570637119},
  {'train/loss': 0.688442611814843,
   'train/accuracy': 0.596150574740577,
   'train/n_examples': 329192,
   'train/learning_rate': 0.001,
   'valid/loss': 0.68442062858273,
   'valid/accuracy': 0.67